# Future Recommendation Model with a different DataSet

##### Made by: WhileTrue
------------------------------------------------------------------------------

In [2]:
# Importing necessary library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

### Load csv data to tables

In [4]:
# books_data = pd.read_csv('books_data.csv')
books_data.columns

NameError: name 'books_data' is not defined

In [ ]:
type(books_data.authors.drop_duplicates)

method

In [ ]:
books_rating = pd.read_csv('books_rating.csv',  nrows=1000000)
books_rating.columns

Index(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/helpfulness',
       'review/score', 'review/time', 'review/summary', 'review/text'],
      dtype='object')

In [ ]:
books_data.rename(columns={"Title": 'title',
                           'previewLink': 'preview_link',
                           'publishedDate': 'published_date',
                           'infoLink': 'info_link',
                           'ratingsCount': 'ratings_count'},inplace=True)
books_data.columns

Index(['title', 'description', 'authors', 'image', 'preview_link', 'publisher',
       'published_date', 'info_link', 'categories', 'ratings_count'],
      dtype='object')

In [ ]:
# showing all cell's content

pd.set_option('display.max_colwidth', None)
books_data.head()['image']

0              http://books.google.com/books/content?id=DykPAAAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api
1    http://books.google.com/books/content?id=IjvHQsCn_pgC&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api
2              http://books.google.com/books/content?id=2tsDAAAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api
3              http://books.google.com/books/content?id=aRSIgJlq6JwC&printsec=frontcover&img=1&zoom=1&source=gbs_api
4                                                                                                                NaN
Name: image, dtype: object

In [ ]:
#Rename of books_rating columns

books_rating.rename(columns={'Id':'id',
                            'Title': 'title',
                            'Price':'price',
                            'User_id':'user_id',
                            'profileName':'profile_name',
                            'review/helpfulness':'review_helpfulness',
                            'review/score':'rating',
                            'review/time':'review_time',
                            'review/summary':'review_summary',
                            'review/text':'review_text'},inplace=True)
books_rating.columns


Index(['id', 'title', 'price', 'user_id', 'profile_name', 'review_helpfulness',
       'rating', 'review_time', 'review_summary', 'review_text'],
      dtype='object')

In [ ]:
books_rating_arranged = books_rating.drop(columns=['review_helpfulness', 'review_time', 'review_summary', 'review_text', 'price'])
books_rating_arranged = books_rating_arranged.dropna(subset=['user_id'])

In [ ]:
#outliers drop: eliminate books with less than n reviews

number_rating = books_rating_arranged.groupby('title')['rating'].count().reset_index()
number_rating.rename(columns={'rating': 'total_ratings'}, inplace = True)

In [ ]:
books_rating_filtered = books_rating_arranged.merge(number_rating ,on='title')

In [ ]:
#books_rating table shape reviews no filtering
books_rating_filtered.shape

(808363, 6)

In [ ]:
books_rating_filtered = books_rating_filtered[books_rating_filtered['total_ratings'] >= 200]

In [ ]:
#books_rating table shape reviews after filtering
books_rating_filtered.shape

(292306, 6)

In [ ]:
books_rating_filtered

,id,title,user_id,profile_name,rating,total_ratings
926,B0007H4QBK,Economics in one lesson,A2FYWUHFF21Q8F,Mark Twian,5.0,314
927,B0007H4QBK,Economics in one lesson,A5P9PV92PRYEE,gary@clearbridge.com,5.0,314
928,B0007H4QBK,Economics in one lesson,A1D4N3NVIN55PK,fmj30cal,4.0,314
929,B0007H4QBK,Economics in one lesson,A34AW9TMV7F69T,GE,3.0,314
930,B0007H4QBK,Economics in one lesson,A1IHQR1NT6CKVR,"Bogey62 ""Bill""",5.0,314
...,...,...,...,...,...,...
805835,084230052X,The Atonement Child,A2YVBNX4Q4CQV8,JoyAnne,5.0,259
805836,084230052X,The Atonement Child,A2WI388LZQM42A,E. Gabriella,5.0,259
805837,084230052X,The Atonement Child,A8POSLLBQUG4V,JB,5.0,259
805838,084230052X,The Atonement Child,A1V8BSW4REJHBN,M.D.C,4.0,259


In [ ]:
books_rating_filtered.drop_duplicates(['user_id','title'], inplace=True)
books_rating_filtered.shape

(245139, 6)

In [ ]:
matrix_ratings = books_rating_filtered.pivot_table(columns='user_id', index='title', values= 'rating')

In [ ]:
matrix_ratings.shape

(526, 147766)

In [ ]:
matrix_ratings.fillna(0, inplace=True)

In [ ]:
matrix_ratings

user_id,A0015610VMNR0JC9XVL1,A00274963RTZUW5BU5ROI,A00538832OF17R8Q8JHTB,A00540411RKGTDNU543WS,A00878773S2MNB00COHKV,A00891092QIVH4W1YP46A,A0092581WFYQNV4KMUZ3,A01023015VSQI0VE22HU,A01038432MVI9JXYTTK5T,A010809536IK2VS9SAU9Q,...,AZZLYAJWAPX91,AZZQV95X90WT7,AZZR4T996J02D,AZZUIE66HZNY1,AZZUTPP7O8M98,AZZVOB0B882KK,AZZVZL4QEHEHO,AZZWKE7JW54GB,AZZXSP27F21T6,AZZYLDF6HREX3
title,,,,,,,,,,,,,,,,,,,,,
"(ESV) English Standard Version Large Print Bible. Premium Bonded Leather, Black, Red Letter Text (English Language)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1491: New Revelations of the Americas Before Columbus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1632 (The Assiti Shards),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"20, 000 Leagues Under the Sea",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights (Riverside editions),0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders (Turtleback School & Library Binding Edition),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
## Authors influention

In [ ]:
title_authors = books_data.drop(columns=['description', 'image', 'preview_link', 'publisher',
       'published_date', 'info_link', 'categories', 'ratings_count'])
title_authors

,title,authors
0,Its Only Art If Its Well Hung!,['Julie Strain']
1,Dr. Seuss: American Icon,['Philip Nel']
2,Wonderful Worship in Smaller Churches,['David R. Ray']
3,Whispers of the Wicked Saints,['Veronica Haddon']
4,"Nation Dance: Religion, Identity and Cultural Difference in the Caribbean",['Edward Long']
...,...,...
212399,The Orphan Of Ellis Island (Time Travel Adventures),['Elvira Woodruff']
212400,Red Boots for Christmas,NaN
212401,Mamaw,['Wild Wild Cabbage']
212402,The Autograph Man,['Zadie Smith']


In [ ]:
## deleting parenthesis, and seperate by commas

title_authors['authors_splited'] = title_authors.authors.str[1:-1].str.split(', ')
title_authors = title_authors.drop(columns=['authors'])
title_authors

,title,authors_splited
0,Its Only Art If Its Well Hung!,['Julie Strain']
1,Dr. Seuss: American Icon,['Philip Nel']
2,Wonderful Worship in Smaller Churches,['David R. Ray']
3,Whispers of the Wicked Saints,['Veronica Haddon']
4,"Nation Dance: Religion, Identity and Cultural Difference in the Caribbean",['Edward Long']
...,...,...
212399,The Orphan Of Ellis Island (Time Travel Adventures),['Elvira Woodruff']
212400,Red Boots for Christmas,NaN
212401,Mamaw,['Wild Wild Cabbage']
212402,The Autograph Man,['Zadie Smith']


In [ ]:
#explode seperate multiple authors
title_authors = title_authors.explode('authors_splited')
title_authors

,title,authors_splited
0,Its Only Art If Its Well Hung!,'Julie Strain'
1,Dr. Seuss: American Icon,'Philip Nel'
2,Wonderful Worship in Smaller Churches,'David R. Ray'
3,Whispers of the Wicked Saints,'Veronica Haddon'
4,"Nation Dance: Religion, Identity and Cultural Difference in the Caribbean",'Edward Long'
...,...,...
212400,Red Boots for Christmas,NaN
212401,Mamaw,'Wild Wild Cabbage'
212402,The Autograph Man,'Zadie Smith'
212403,Student's Solutions Manual for Johnson/Mowry's Mathematics: A Practical Odyssey,'David B. Johnson'


In [ ]:
title_authors

,title,authors_splited
0,Its Only Art If Its Well Hung!,'Julie Strain'
1,Dr. Seuss: American Icon,'Philip Nel'
2,Wonderful Worship in Smaller Churches,'David R. Ray'
3,Whispers of the Wicked Saints,'Veronica Haddon'
4,"Nation Dance: Religion, Identity and Cultural Difference in the Caribbean",'Edward Long'
...,...,...
212400,Red Boots for Christmas,NaN
212401,Mamaw,'Wild Wild Cabbage'
212402,The Autograph Man,'Zadie Smith'
212403,Student's Solutions Manual for Johnson/Mowry's Mathematics: A Practical Odyssey,'David B. Johnson'


In [ ]:
#pd.pivot_table(title_authors, values="authors_splited")

In [ ]:
#merging ratings with authors

matrix_ratings_authors = matrix_ratings.merge(title_authors, on='title')
matrix_ratings_authors = matrix_ratings_authors.set_index('title')
matrix_ratings_authors

,A0015610VMNR0JC9XVL1,A00274963RTZUW5BU5ROI,A00538832OF17R8Q8JHTB,A00540411RKGTDNU543WS,A00878773S2MNB00COHKV,A00891092QIVH4W1YP46A,A0092581WFYQNV4KMUZ3,A01023015VSQI0VE22HU,A01038432MVI9JXYTTK5T,A010809536IK2VS9SAU9Q,...,AZZQV95X90WT7,AZZR4T996J02D,AZZUIE66HZNY1,AZZUTPP7O8M98,AZZVOB0B882KK,AZZVZL4QEHEHO,AZZWKE7JW54GB,AZZXSP27F21T6,AZZYLDF6HREX3,authors_splited
title,,,,,,,,,,,,,,,,,,,,,
"(ESV) English Standard Version Large Print Bible. Premium Bonded Leather, Black, Red Letter Text (English Language)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,'Crossway Bibles'
1491: New Revelations of the Americas Before Columbus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,'Charles C. Mann'
1632 (The Assiti Shards),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,'Eric Flint'
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,'George Orwell'
"20, 000 Leagues Under the Sea",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,'Jules Verne'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights (Riverside editions),0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,'Emily Brontë'
Year of Wonders (Turtleback School & Library Binding Edition),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,'Arthur Pike & David Pike'
Zen and the Art of Motorcycle Maintenance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,'Robert M. Pirsig'


In [ ]:
pattern = r'authors\w+'
#final_matrix = final_matrix.drop(list(final_matrix.filter(regex=pattern)))
authors_matrix = matrix_ratings_authors.filter(regex=pattern)
authors_matrix

,authors_splited
title,
"(ESV) English Standard Version Large Print Bible. Premium Bonded Leather, Black, Red Letter Text (English Language)",'Crossway Bibles'
1491: New Revelations of the Americas Before Columbus,'Charles C. Mann'
1632 (The Assiti Shards),'Eric Flint'
1984,'George Orwell'
"20, 000 Leagues Under the Sea",'Jules Verne'
...,...
Wuthering Heights (Riverside editions),'Emily Brontë'
Year of Wonders (Turtleback School & Library Binding Edition),'Arthur Pike & David Pike'
Zen and the Art of Motorcycle Maintenance,'Robert M. Pirsig'


In [ ]:

authors_matrix = pd.get_dummies(authors_matrix)

In [ ]:
authors_matrix = authors_matrix.groupby('title').sum()
authors_matrix = authors_matrix.mul(20)

In [ ]:
authors_matrix

,"authors_splited_""Alex Prud'homme""","authors_splited_""Ellen O'Connor""","authors_splited_""Kate O'Beirne""",authors_splited_'A. J. Russell',authors_splited_'Abraham Verghese',authors_splited_'Adam Smith',authors_splited_'Aerie Books Ltd',authors_splited_'Agatha Christie',authors_splited_'Alan Paton',authors_splited_'Aldous Huxley',...,authors_splited_'William Manchester',authors_splited_'William Queen',authors_splited_'William S. Burroughs',authors_splited_'William Shakespeare',authors_splited_'Óscar Wilde',authors_splited_Booksellers,authors_splited_Chicago',authors_splited_Firm,authors_splited_Inc',authors_splited_Jr.'
title,,,,,,,,,,,,,,,,,,,,,
"(ESV) English Standard Version Large Print Bible. Premium Bonded Leather, Black, Red Letter Text (English Language)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1491: New Revelations of the Americas Before Columbus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1632 (The Assiti Shards),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1984,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"20, 000 Leagues Under the Sea",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights (Riverside editions),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Year of Wonders (Turtleback School & Library Binding Edition),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zen and the Art of Motorcycle Maintenance,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
matrix_ratings_authors.drop(columns=['authors_splited'])

,A0015610VMNR0JC9XVL1,A00274963RTZUW5BU5ROI,A00538832OF17R8Q8JHTB,A00540411RKGTDNU543WS,A00878773S2MNB00COHKV,A00891092QIVH4W1YP46A,A0092581WFYQNV4KMUZ3,A01023015VSQI0VE22HU,A01038432MVI9JXYTTK5T,A010809536IK2VS9SAU9Q,...,AZZLYAJWAPX91,AZZQV95X90WT7,AZZR4T996J02D,AZZUIE66HZNY1,AZZUTPP7O8M98,AZZVOB0B882KK,AZZVZL4QEHEHO,AZZWKE7JW54GB,AZZXSP27F21T6,AZZYLDF6HREX3
title,,,,,,,,,,,,,,,,,,,,,
"(ESV) English Standard Version Large Print Bible. Premium Bonded Leather, Black, Red Letter Text (English Language)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1491: New Revelations of the Americas Before Columbus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1632 (The Assiti Shards),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"20, 000 Leagues Under the Sea",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights (Riverside editions),0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders (Turtleback School & Library Binding Edition),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
final_matrix = matrix_ratings_authors.merge(authors_matrix, on='title')

In [ ]:
final_matrix

,A0015610VMNR0JC9XVL1,A00274963RTZUW5BU5ROI,A00538832OF17R8Q8JHTB,A00540411RKGTDNU543WS,A00878773S2MNB00COHKV,A00891092QIVH4W1YP46A,A0092581WFYQNV4KMUZ3,A01023015VSQI0VE22HU,A01038432MVI9JXYTTK5T,A010809536IK2VS9SAU9Q,...,authors_splited_'William Manchester',authors_splited_'William Queen',authors_splited_'William S. Burroughs',authors_splited_'William Shakespeare',authors_splited_'Óscar Wilde',authors_splited_Booksellers,authors_splited_Chicago',authors_splited_Firm,authors_splited_Inc',authors_splited_Jr.'
title,,,,,,,,,,,,,,,,,,,,,
"(ESV) English Standard Version Large Print Bible. Premium Bonded Leather, Black, Red Letter Text (English Language)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1491: New Revelations of the Americas Before Columbus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1632 (The Assiti Shards),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
"20, 000 Leagues Under the Sea",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights (Riverside editions),0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
Year of Wonders (Turtleback School & Library Binding Edition),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
Zen and the Art of Motorcycle Maintenance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# select numeric columns
numeric_cols = final_matrix.select_dtypes(include=[float, int]).columns

# drop non-numeric columns
final_matrix = final_matrix[numeric_cols]

In [ ]:
final_matrix = final_matrix.astype(int)
final_matrix

,A0015610VMNR0JC9XVL1,A00274963RTZUW5BU5ROI,A00538832OF17R8Q8JHTB,A00540411RKGTDNU543WS,A00878773S2MNB00COHKV,A00891092QIVH4W1YP46A,A0092581WFYQNV4KMUZ3,A01023015VSQI0VE22HU,A01038432MVI9JXYTTK5T,A010809536IK2VS9SAU9Q,...,AZZLYAJWAPX91,AZZQV95X90WT7,AZZR4T996J02D,AZZUIE66HZNY1,AZZUTPP7O8M98,AZZVOB0B882KK,AZZVZL4QEHEHO,AZZWKE7JW54GB,AZZXSP27F21T6,AZZYLDF6HREX3
title,,,,,,,,,,,,,,,,,,,,,
"(ESV) English Standard Version Large Print Bible. Premium Bonded Leather, Black, Red Letter Text (English Language)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1491: New Revelations of the Americas Before Columbus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1632 (The Assiti Shards),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1984,0,0,0,0,0,0,0,0,0,0,...,0,5,0,0,0,0,0,0,0,0
"20, 000 Leagues Under the Sea",0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights (Riverside editions),0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Year of Wonders (Turtleback School & Library Binding Edition),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zen and the Art of Motorcycle Maintenance,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#index = final_matrix.index.get_loc('Jane Eyre (Signet classics)')
#print(index)


In [ ]:
model = None
def recommendation_model_rat_auth(book_title, num_neighbours) :
    book_sparse = csr_matrix(final_matrix)
    model = NearestNeighbors(algorithm= 'brute')
    model.fit(book_sparse)
    index = final_matrix.index.get_loc(book_title)
    suggestion = model.kneighbors(final_matrix.iloc[index,:].values.reshape(1,-1), n_neighbors=num_neighbours )[1]
    book_titles = [] 
    for i in suggestion:
        book_titles.append(final_matrix.index[i])   

    final_result = pd.DataFrame({"title": book_titles[0].tolist()})
    final_result = final_result.merge(books_data, on="title")  
    
    return final_result

In [ ]:
output = recommendation_model_rat_auth('20, 000 Leagues Under the Sea' , 20)
#output


,title,description,authors,image,preview_link,publisher,published_date,info_link,categories,ratings_count
0,"20, 000 Leagues Under the Sea","The year 1866 was signalised by a remarkable incident, a mysterious and puzzling phenomenon, which doubtless no one has yet forgotten. Not to mention rumours which agitated the maritime population and excited the public mind, even in the interior of continents, seafaring men were particularly excited. Merchants, common sailors, captains of vessels, skippers, both of Europe and America, naval officers of all countries, and the Governments of several States on the two continents, were deeply interested in the matter. For some time past vessels had been met by ""an enormous thing,"" a long object, spindle-shaped, occasionally phosphorescent, and infinitely larger and more rapid in its movements than a whale. The facts relating to this apparition (entered in various log-books) agreed in most respects as to the shape of the object or creature in question, the untiring rapidity of its movements, its surprising power of locomotion, and the peculiar life with which it seemed endowed. If it was a whale, it surpassed in size all those hitherto classified in science. Taking into consideration the mean of observations made at divers times—rejecting the timid estimate of those who assigned to this object a length of two hundred feet, equally with the exaggerated opinions which set it down as a mile in width and three in length—we might fairly conclude that this mysterious being surpassed greatly all dimensions admitted by the learned ones of the day, if it existed at all. And that it DID exist was an undeniable fact; and, with that tendency which disposes the human mind in favour of the marvellous, we can understand the excitement produced in the entire world by this supernatural apparition. As to classing it in the list of fables, the idea was out of the question. On the 20th of July, 1866, the steamer Governor Higginson, of the Calcutta and Burnach Steam Navigation Company, had met this moving mass five miles off the east coast of Australia. Captain Baker thought at first that he was in the presence of an unknown sandbank; he even prepared to determine its exact position when two columns of water, projected by the mysterious object, shot with a hissing noise a hundred and fifty feet up into the air. Now, unless the sandbank had been submitted to the intermittent eruption of a geyser, the Governor Higginson had to do neither more nor less than with an aquatic mammal, unknown till then, which threw up from its blow-holes columns of water mixed with air and vapour. Similar facts were observed on the 23rd of July in the same year, in the Pacific Ocean, by the Columbus, of the West India and Pacific Steam Navigation Company. But this extraordinary creature could transport itself from one place to another with surprising velocity; as, in an interval of three days, the Governor Higginson and the Columbus had observed it at two different points of the chart, separated by a distance of more than seven hundred nautical leagues.",['Jules Verne'],http://books.google.com/books/content?id=LGIiDQAAQBAJ&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api,"http://books.google.com/books?id=LGIiDQAAQBAJ&printsec=frontcover&dq=20,+000+Leagues+Under+the+Sea&hl=&cd=1&source=gbs_api",Library of Alexandria,2020-09-28,https://play.google.com/store/books/details?id=LGIiDQAAQBAJ&source=gbs_api,NaN,NaN
1,"Twenty thousand leagues under the sea, (The Windermere series)",NaN,['Jules Verne'],NaN,"http://books.google.com/books?id=a6X8zgEACAAJ&dq=Twenty+thousand+leagues+under+the+sea,+(The+Windermere+series)&hl=&cd=1&source=gbs_api",NaN,1930,"http://books.google.com/books?id=a6X8zgEACAAJ&dq=Twenty+thousand+leagues+under+the+sea,+(The+Windermere+series)&hl=&source=gbs_api",NaN,NaN
2,Twenty Thousand Leagues Under the Sea (Caxton Edition),NaN,"['McClurg, Firm, Booksellers, Chicago']",http://books.google.com/books/content?id=pUI0A

In [ ]:
book_names = books_data['title'].tolist()
authors = title_authors.reset_index()['authors_splited'].drop_duplicates()
authors

0               'Julie Strain'
1                 'Philip Nel'
2               'David R. Ray'
3            'Veronica Haddon'
4                'Edward Long'
                  ...         
264090    'Patrick J. Harbula'
264091         'Daniel Banach'
264095     'Wild Wild Cabbage'
264097      'David B. Johnson'
264098       'Thomas A. Mowry'
Name: authors_splited, Length: 153971, dtype: object

In [ ]:
import pickle

pickle.dump(model, open('artifacts/model.pkl','wb'))
pickle.dump(book_names, open('artifacts/book_names.pkl','wb'))
pickle.dump(final_matrix,open('artifacts/book_pivot.pkl','wb'))
pickle.dump(authors,open('artifacts/authors.pkl','wb'))

In [ ]:
#title category table set

title_category = books_data.drop(columns=['description', 'image', 'preview_link', 'publisher',
       'published_date', 'info_link', 'authors', 'ratings_count'])
title_category = title_category.set_index('title')
title_category = title_category.dropna()

In [ ]:
#table category to dummies

title_category['categories_splited'] = title_category.categories.str[1:-1].str.split(' & ')
title_category = title_category.drop(columns=['categories'])
title_category

,categories_splited
title,
Its Only Art If Its Well Hung!,"['Comics, Graphic Novels']"
Dr. Seuss: American Icon,"['Biography, Autobiography']"
Wonderful Worship in Smaller Churches,['Religion']
Whispers of the Wicked Saints,['Fiction']
The Church of Christ: A Biblical Ecclesiology for Today,['Religion']
...,...
The Magic of the Soul: Applying Spiritual Power to Daily Living,"['Body, Mind, Spirit']"
Autodesk Inventor 10 Essentials Plus,['Computers']
The Orphan Of Ellis Island (Time Travel Adventures),['Juvenile Fiction']


In [ ]:
#final title category table
title_category = title_category.explode('categories_splited')
title_category

,categories_splited
title,
Its Only Art If Its Well Hung!,'Comics
Its Only Art If Its Well Hung!,Graphic Novels'
Dr. Seuss: American Icon,'Biography
Dr. Seuss: American Icon,Autobiography'
Wonderful Worship in Smaller Churches,'Religion'
...,...
The Magic of the Soul: Applying Spiritual Power to Daily Living,Spirit'
Autodesk Inventor 10 Essentials Plus,'Computers'
The Orphan Of Ellis Island (Time Travel Adventures),'Juvenile Fiction'


In [ ]:
#rating average table

rating_count = books_data.drop(columns=['description', 'image', 'preview_link', 'publisher',
       'published_date', 'info_link', 'authors', 'categories'])
rating_count = rating_count.set_index('title')
rating_count = rating_count.dropna()
rating_count

,ratings_count
title,
The Church of Christ: A Biblical Ecclesiology for Today,5.0
A Walk in the Woods: a Play in Two Acts,3.0
Voices from the Farm: Adventures in Community Living,1.0
The Battleship Bismarck,1.0
Mini-mysteries,1.0
...,...
Thin Within,9.0
The Awakening and Selected Stories (Modern Library Classics),2.0
Final things,4.0


In [ ]:
rating_count = rating_count.drop(rating_count[rating_count['ratings_count'] <= 50].index)

In [ ]:
book_categories = rating_count.merge(title_category, on='title')
book_categories

,ratings_count,categories_splited
title,,
The Best Short Stories of Edgar Allan Poe,1180.0,'Fiction'
The Cruelest Months,84.0,'Fiction'
Helter Skelter,80.0,'True Crime'
Small Gods,85.0,'Fiction'
Hard Times,64.0,'Fiction'
...,...,...
"I Danced on the Sand: A Memoir of Child Abuse, Hope, and Promise",153.0,'Biography
"I Danced on the Sand: A Memoir of Child Abuse, Hope, and Promise",153.0,Autobiography'
Progeny-A Novel,93.0,'Fiction'


In [ ]:
#dummies recommended categories

content_based = pd.get_dummies(book_categories).groupby('title').sum().drop(columns=['ratings_count'])
content_based = content_based.merge(rating_count, on='title').sort_values(by='ratings_count', ascending=False)
content_based

,"categories_splited_""Children's poetry, American""","categories_splited_""Children's stories""","categories_splited_'Abnormalities, Human'",categories_splited_'Adultery',categories_splited_'Adventure and adventurers',categories_splited_'Adventure stories',categories_splited_'African American boys',categories_splited_'African Americans',categories_splited_'Alienation (Social psychology)',categories_splited_'Alligators',...,categories_splited_Autobiography',categories_splited_Disciplines',categories_splited_Economics',categories_splited_Fitness',categories_splited_Graphic Novels',categories_splited_RELATIONSHIPS',categories_splited_Recreation',categories_splited_Relationships',categories_splited_Spirit',ratings_count
title,,,,,,,,,,,,,,,,,,,,,
The Alchemist,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4895.0
Unbroken,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,4572.0
A Mission to Millions; The Story of Ernie Allen and the Every Home Crusade,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,4572.0
Blue Like Jazz: Nonreligious Thoughts on Christian Spirituality,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,4562.0
Eclipse,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4392.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The mother,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,51.0
Narrative of the Life of Frederick Douglass,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,51.0
Winesboro Ohio (Collected Works of Sherwood Anderson),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,51.0


In [ ]:
def suggestion_not_the_best_ones(book_title, num_neighbours) :
    book_sparse = csr_matrix(content_based)
    model = NearestNeighbors(algorithm='brute')
    model.fit(book_sparse)
    index = content_based.index.get_loc(book_title)
    print(index)
    suggestion = model.kneighbors(content_based.iloc[index,:].values.reshape(1,-1), n_neighbors=num_neighbours )[1]
    book_titles = [] 
    for i in suggestion:
        book_titles.append(content_based.index[i])
    return book_titles

In [ ]:
suggestion_not_the_best_ones('Unbroken', 6)

1


[Index(['Unbroken',
        'A Mission to Millions; The Story of Ernie Allen and the Every Home Crusade',
        'Blue Like Jazz: Nonreligious Thoughts on Christian Spirituality',
        'Eclipse', 'Redeeming Love', 'The Alchemist'],
       dtype='object', name='title')]